# Exploring the *Ruegeria pomeroyi* FBA model

In [1]:
# Necessary imports
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import escher
from escher import Builder
import cobra

# Load model
model = cobra.io.read_sbml_model("../base_model/Rpom_06.xml")


Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


> ### From Matteo:
>
> 1) The different models vary in their biomass composition, with the number corresponding to the growth rate across different nutrient sources according to out lab's measurements. For example, Rpom_055 has the biomass extrapolated at a growth rate 0.55/h.
> 2) I really don't have much to add compared to what I told Arion above - it is mostly to make sure that the pathways are thermodynamically consistent and that the atp/nadh, as well as the individual biomass components are produced in the expected pathways.
> 3) In addition, there are a few genes to be added to the model (I have added three reactions compared to what Arion did: PDH, PHB-SYNTHESIS, PHB-DEGRADATION). Here are the uniprot IDs of the corresponding proteins:
>  - Pyruvate dehydrogenase (PDH):
>     - Q5LR87
>     - Q5LR88
>     - Q5LR89
>     - Q5LKR2
>   - PHB synthesis (including phasin)
>     - Q5LTW8
>     - Q5LTW7
>   - PHB degradation
>     - Q5LTW9
> 
> The problem is that the model was made using a previous Biocyc genome, with different gene names. I don't know whether this older annotation is still available. So, it would be necessary to update the gene IDs to make sure that all genes are properly annotated.

## Issue: Which BioCyc database?

Ruegeria pomeroyi DSS-3
<img src="img/Ruegeria%20pomeroyi%20DSS-3.png" width=250/>

Ruegeria pomeroyi DSS-3 representative genome
<img src="img/Ruegeria%20pomeroyi%20DSS-3%20representative%20genome.png" width=250/>

In [ ]:
model.summary()

# Exploring the Model

In [2]:
model

Name,Rpom_06
Memory address,0x07f86a870bfa0
Number of metabolites,1657
Number of reactions,1929
Number of groups,0
Objective expression,1.0*EX_glc - 1.0*EX_glc_reverse_8e37d
Compartments,"c, p, e"


In [ ]:
model.objective.expression

In [ ]:
solution = model.optimize()
solution

In [ ]:
(solution.shadow_prices).sort_values(ascending=False)[:25]


In [ ]:
objbuilder = Builder()
objbuilder.model = model
objbuilder.reaction_data = solution.fluxes
objbuilder

Genes are missing (!)

In [3]:
model.genes

[]

Thermodynamic consistency is apparently an issue with the base model. Begin exploring by counting bounds on reaction fluxes:

In [ ]:
bounds = np.zeros((2, len(model.reactions)))

for c, reaction in enumerate(model.reactions):
    bounds[:, c] = reaction.lower_bound, reaction.upper_bound

bounds_counter = Counter([tuple(bounds[:, c]) for c in range(bounds.shape[1])])
bounds_counter = list(bounds_counter.items())

bounds_counter.sort(key=lambda x: abs(x[0][1] - x[0][0]))

fig, ax = plt.subplots()
for y, ((lb, ub), count) in enumerate(bounds_counter):
    ax.hlines(y, lb, ub, color="k")
    ax.scatter([lb, ub], [y, y])
    middle = (lb + ub) / 2
    ax.text(middle, y + 0.1,
            f"{count} reaction(s)",
            horizontalalignment="right" if middle > 0 else "left")
ax.set_xscale("symlog")
ax.set_ybound(-1, y + 1)
ax.set_yticks(list(range(y + 1)))
ax.set_yticklabels([bounds for bounds, _ in bounds_counter])
ax.set_xlabel("Flux (mmol / [gDW h])")
ax.set_ylabel("Constraints")
fig.tight_layout()


Get reactions that involve NADH:

In [ ]:
nadh = model.metabolites.get_by_id("NADH[c]")
nadh_reactions = model.reactions.query(
    lambda m: nadh in m, attribute="metabolites")
len(nadh_reactions)

Can we build a Escher diagram containing all reactions affecting NADH?

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.escher_utils.mapbuilder import MapBuilder

In [ ]:
mapbuilder = MapBuilder(template="../utils/escher_utils/template.json",
                        name="nadh_reaction_map")

for rxn in nadh_reactions:
    mapbuilder.add_reaction(rxn)

mapbuilder.write("./nadh_reactions_map.json")

Need to load the map manually using Map>Load map JSON:

In [ ]:
escher.list_available_maps()